In [1]:
import sys
import requests
import pandas as pd
import json
import boto3
import botocore
import pyarrow
import numpy
import snowflake.connector
from datetime import datetime
from config import api_key_secret, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY
from aws_secretsmanager_caching import SecretCache, SecretCacheConfig 

# Get secrets from AWS Secrets Manager
client = botocore.session.get_session().create_client('secretsmanager')
cache_config = SecretCacheConfig()
cache = SecretCache( config = cache_config, client = client)
secrets = cache.get_secret_string('snowflake-creds')
secrets = (json.loads(secrets)) #Put secrets into a dictionary

# Initialize connection to Snowflake
ctx = snowflake.connector.connect(
    user='NICKLILLEYMAN',
    password=(secrets['password']),
    account=(secrets['account']),
    region =(secrets['region']),
    warehouse=(secrets['warehouse']),
    database='SBX_RAW',
    schema='PUBLIC'
    )

# Create a cursor object.
cur = ctx.cursor()

# Execute a statement that will generate a result set.
sql = ("""
SELECT
     DIM_SUBURB_SK
    ,SUBURB_ID
    ,SUBURB
    ,POSTCODE
    ,STATE
FROM SBX_ANALYTICS.DBT_NLILLEYMAN_COMMON.DIM_SUBURB_GEOGRAPHY
WHERE
     STATE = 'WA'
AND  DIM_SUBURB_SK NOT IN (SELECT SEED_KEY FROM SBX_RAW.PUBLIC.API_CALL_LOG)
AND DIM_SUBURB_SK = '009658a289fa245230ce5a9c4b24ba77'
ORDER BY DIM_SUBURB_SK
 """)
cur.execute(sql)

#Put query results into a dataframe
df = cur.fetch_pandas_all()
df['row_num'] = df.reset_index().index
print(df)

                      DIM_SUBURB_SK              SUBURB_ID         SUBURB  \
0  009658a289fa245230ce5a9c4b24ba77  Bassendean Dc-6054-WA  Bassendean Dc   

  POSTCODE STATE  row_num  
0     6054    WA        0  
